In [1]:
import argparse
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding as as_padding
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_private_key
import os
import random

### 1. Генерация ключей гибридной системы

In [4]:
way = os.path.abspath('')
file_settings= {
    'initial_file': way + '\\' + 'initial_file.txt',
    'encrypted_file': way + '\\' + 'encrypted_file.txt',
    'decrypted_file': way + '\\' + 'decrypted_file.txt',
    'symmetric_key': way + '\\' + 'symmetric_key.txt',
    'public_key': way + '\\' + 'public_key.pem',
    'private_key': way + '\\' + 'private_key.pem',
}

keys = [i for i in range(5, 17, 1)]
iv = os.urandom(8)


генерация ключа для симметричного алгоритма

In [5]:
def gen_keys(file_settings: dict, keys: list):
    len_key = random.randint(0, len(keys) - 1)
    key = os.urandom(keys[len_key])
    print('Ключ симметричного шифрования:')
    print(type(key))
    print(key)
    
    assym_keys = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    private_key = assym_keys
    public_key = assym_keys.public_key()
    
    print('\nЗакрытый ключ ассимметричного шифрования:')
    print(type(private_key))
    print(private_key)
    print('\nОтрытый ключ ассимметричного шифрования:')
    print(type(public_key))
    print(public_key)
    
    c_key = public_key.encrypt(key, as_padding.OAEP(mgf=as_padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    print('\nЗашифрованный ключ симметричного шифрования')
    print(c_key)
    
    with open(file_settings['public_key'], 'wb') as public_out:
            public_out.write(public_key.public_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo))

    with open(file_settings['private_key'], 'wb') as private_out:
            private_out.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()))
            
    with open(file_settings['symmetric_key'], 'wb') as key_file:
        key_file.write(c_key)

генерация пары ключей для acсимметричного алгоритма

In [6]:
gen_keys(file_settings, keys)

Ключ симметричного шифрования:
<class 'bytes'>
b'/\x9c\x05\xd4:/\xcb\xf6\x89\x90F'

Закрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey'>

Отрытый ключ ассимметричного шифрования:
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPublicKey'>

Зашифрованный ключ симметричного шифрования
b'\rS\r\x1f\xc3y\xed\xa8\xc7\xb2\x9f\xc66\xe98\xb9\xedw\x9b\x0e\x9c\xf4EO\x83\xea\xad\xc0\xef\xf0*\x1e\x89%d \xce\xf8\xf5,m\xf2\xd6\xa5\xe2\x1f\xd6\x92@\x8a-\xa3lr\xc3\'\xd6\x0c\xfa{\x1f\xe02\x811(\x11\xcc\xdfSs\xff\xa9z@\x95\x85\x84\x02/\x10lL\xdc\xbb\xb1\xae\x97\xba\xf3\x10\x8f\x84\xb0\n\xc3J\xa5\xff\x00\x0e}\xb0^\xc6\x9f9\xebc\xe3\xa5\x83\x86L\xd8\x1f\xf5\xd4\x89\xd9 \xa5\x91\x94\xd8uH/\x13\x90\x076\xb7\xac/\xa5}L\xfaY\x14PC\xb7\x8c\xad\r\xda\xc5\x15U\xa8\\\\\xb8\xacK\xeb\x93l\xfc(\xcf\x91\x06\xea\xb0\xcc\xc7\xacxfU\xd4$p**Sg\x03\xe3\xee\x81t\xcb\xe8K`\xd4\xef\x12"\xdd\x99}\x8bQ\x80d\xe66D\x14\xb1#\xe2\x8d\xafxi\xbbK1\x95\xf6\xd0Y\xad\x93\xfc\xe

### 2. Шифрование данных гибридной системой

In [7]:
def sym_key_decryption(file_settings: dict):
    with open(file_settings['private_key'], 'rb') as file:
        private_bytes = file.read()
    d_private_key = load_pem_private_key(private_bytes,password=None,)
    print('\nЗасшифрованный ассимметричный ключ:', d_private_key)

    with open(file_settings['symmetric_key'], 'rb') as file:
        sym_key = file.read()
        
    print('Ключ симметричного шифрования:', sym_key)
  
    dec_sym_key = d_private_key.decrypt(sym_key,as_padding.OAEP(mgf=as_padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
    
    print('\nРасшифрованный люч симметричного шифрования:', dec_sym_key)
    
    return dec_sym_key
    
    

In [8]:
def text_encryption(file_settings: dict):
    with open(file_settings['initial_file'], 'r', encoding = 'UTF-8') as file:
        text = file.read()
    print('Исходный текст:\n', text)
    
    dec_sym_key = sym_key_decryption(file_settings)
    
    padder = sym_padding.ANSIX923(32).padder()
    padded_text = padder.update(bytes(text, 'UTF-8'))+padder.finalize()
    
    cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    enc_text = encryptor.update(padded_text) + encryptor.finalize()
    
    print('зашифрованный текст')
    print(enc_text)
    
    with open(file_settings['encrypted_file'], 'wb') as file:
        file.write(enc_text)

In [9]:
text_encryption(file_settings)

Исходный текст:
 КОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ Мы, многонациональный народ Российской Федерации, соединенные общей судьбой на своей земле, утверждая права и свободы человека, гражданский мир и согласие, сохраняя исторически сложившееся государственное единство, исходя из общепризнанных принципов равноправия и самоопределения народов, чтя память предков, передавших нам любовь и уважение к Отечеству, веру в добро и справедливость, возрождая суверенную государственность России и утверждая незыблемость ее демократической основы, стремясь обеспечить благополучие и процветание России, исходя из ответственности за свою Родину перед нынешним и будущими поколениями, сознавая себя частью мирового сообщества, принимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.

Засшифрованный ассимметричный ключ: <cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey object at 0x000001A5C8E5ACA0>
Ключ симметричного шифрования: b'\rS\r\x1f\xc3y\xed\xa8\xc7\xb2\x9f\xc66\xe98\xb9\xedw\x9b\x0e\x9c\xf4EO\x83\xea\xad\xc0\

C:\Users\Marishka\AppData\Local\Temp\ipykernel_3360\3719966415.py:11: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))


### 3. Дешифрование данных гибридной системой

In [12]:
def text_decryption(file_settings: dict):
    with open(file_settings['encrypted_file'], 'rb') as file:
        enc_text = file.read()
    #print('Зашифрованный текст:\n', enc_text)
    
    dec_sym_key = sym_key_decryption(file_settings)
    cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))
    
    decryptor = cipher.decryptor()
    dc_text = decryptor.update(enc_text) + decryptor.finalize()

    unpadder = sym_padding.ANSIX923(32).unpadder()
    unpadded_dc_text = unpadder.update(dc_text) + unpadder.finalize()

    print(dc_text.decode('UTF-8'))
    print('----------------'*5)
    print(unpadded_dc_text.decode('UTF-8'))

In [13]:
text_decryption(file_settings)


Засшифрованный ассимметричный ключ: <cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey object at 0x000001A5C9D30F40>
Ключ симметричного шифрования: b'\rS\r\x1f\xc3y\xed\xa8\xc7\xb2\x9f\xc66\xe98\xb9\xedw\x9b\x0e\x9c\xf4EO\x83\xea\xad\xc0\xef\xf0*\x1e\x89%d \xce\xf8\xf5,m\xf2\xd6\xa5\xe2\x1f\xd6\x92@\x8a-\xa3lr\xc3\'\xd6\x0c\xfa{\x1f\xe02\x811(\x11\xcc\xdfSs\xff\xa9z@\x95\x85\x84\x02/\x10lL\xdc\xbb\xb1\xae\x97\xba\xf3\x10\x8f\x84\xb0\n\xc3J\xa5\xff\x00\x0e}\xb0^\xc6\x9f9\xebc\xe3\xa5\x83\x86L\xd8\x1f\xf5\xd4\x89\xd9 \xa5\x91\x94\xd8uH/\x13\x90\x076\xb7\xac/\xa5}L\xfaY\x14PC\xb7\x8c\xad\r\xda\xc5\x15U\xa8\\\\\xb8\xacK\xeb\x93l\xfc(\xcf\x91\x06\xea\xb0\xcc\xc7\xacxfU\xd4$p**Sg\x03\xe3\xee\x81t\xcb\xe8K`\xd4\xef\x12"\xdd\x99}\x8bQ\x80d\xe66D\x14\xb1#\xe2\x8d\xafxi\xbbK1\x95\xf6\xd0Y\xad\x93\xfc\xef\xee\xa6 \xb3\xd5\xd65\xb0\xc4\x00\xc74\xe6\x1d\xfc\xfa\xa7~i=\\\r\x97yM\xe5%\xd8B\xb3\x87F\xe4\xb8'

Расшифрованный люч симметричного шифрования: b'/\x9c\x05\xd4:/\xcb\xf6\x89\x90F'
КОНСТ

C:\Users\Marishka\AppData\Local\Temp\ipykernel_3360\2486391256.py:7: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher = Cipher(algorithms.CAST5(dec_sym_key), modes.CBC(iv))
